In [1]:
import osmgraph
import geog
import networkx as nx
import geojsonio
import json
import itertools
from astar import A_star 
from time import time

In [2]:
# filename = '/Users/grigoriipogorelov/Downloads/map_inno.osm'
filename = '/Users/grigoriipogorelov/Downloads/map_kazan.osm'
g = osmgraph.parse_file(filename)

valid_nodes = {}
for n1, n2 in g.edges():
    valid_nodes[n1] = 0
    valid_nodes[n2] = 0
    c1, c2 = osmgraph.tools.coordinates(g, (n1, n2))
    g[n1][n2]['length'] = geog.distance(c1, c2)

In [8]:
src = 1813517414
dst = 1233424921

k_range = 2
a_star = A_star(g, valid_nodes)
start = time()
a_star_path, _ = a_star.a_star_search(src,dst, k_range)
end = time()
print('path search time is %.1f min' % ((end-start)/60))
coords = osmgraph.tools.coordinates(g, a_star_path)
path = nx.shortest_path(g, src, dst, 'length')
coords2 = osmgraph.tools.coordinates(g, path)

# geojsonio.display(json.dumps({'type': 'LineString', 'coordinates': coords}))
geojsonio.display(json.dumps([{'type': 'LineString', 'coordinates': coords2},{'type': 'LineString', 'coordinates': coords}]))

path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
path search time is 0.0 min
